# Save DTA Draft

Saves a new draft version for a DTA using SCD Type 2.

**Operation:** `SAVE_DRAFT`

**What it does:**
- Closes the current draft (sets `is_current=FALSE`, `effective_end_ts=NOW()`)
- Creates new draft records with incremented version (e.g., draft1 -> draft2)
- Updates `md_version_registry` (old draft -> SUPERSEDED, new draft -> ACTIVE)

**Parameters:**
- `dta_id` - DTA identifier (required)
- `library_type` - Library type (e.g., transfer_variables)

**Note:** Changes data should be provided via a staging table or API call.


In [ ]:
# Cell 1: Imports
import json
from clinical_data_standards_framework.versioning import (
    save_dta_draft,
    get_dta_current_version,
    get_version_registry_schema
)

print("="*80)
print("SAVE DTA DRAFT")
print("="*80)


In [ ]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
versioning_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="versioning"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

catalog = globals_dict['catalog']
gold_schema = globals_dict['gold_schema']

# Versioning configuration from YAML
library_tables = versioning_dict.get('library_tables', [])
registry_table_name = versioning_dict.get('registry_table', 'md_version_registry')

print(f"Catalog: {catalog}")
print(f"Gold Schema: {gold_schema}")
print(f"Versioning Config: {len(library_tables)} library table(s)")
print(f"Created by: {created_by_principal}")


In [ ]:
# Cell 3: Get Widget Parameters (job-level parameters only)
dbutils.widgets.text("dta_id", "", "DTA ID")
dbutils.widgets.text("library_type", "", "Library Type (defaults from config)")

dta_id = dbutils.widgets.get("dta_id")
library_type_param = dbutils.widgets.get("library_type")

# Default to first library type from config if not specified
if library_type_param:
    library_type = library_type_param
elif library_tables:
    library_type = library_tables[0].get('library_type', 'transfer_variables')
else:
    library_type = 'transfer_variables'

print(f"\nDTA ID: {dta_id}")
print(f"Library Type: {library_type} (from {'param' if library_type_param else 'config'})")

# Validate
if not dta_id:
    raise ValueError("dta_id parameter is required")


In [ ]:
# Cell 4: Configure Table Names (from config)
# Build library table map from config
library_table_map = {}
for lib_config in library_tables:
    lib_type = lib_config.get('library_type')
    lib_name = lib_config.get('name')
    lib_schema = lib_config.get('schema', gold_schema)
    if lib_type and lib_name:
        library_table_map[lib_type] = f"{catalog}.{lib_schema}.{lib_name}"

if library_type not in library_table_map:
    available = ", ".join(library_table_map.keys())
    raise ValueError(f"Unknown library_type: {library_type}. Available: {available}")

library_table = library_table_map[library_type]
registry_table = f"{catalog}.{gold_schema}.{registry_table_name}"

print(f"\nLibrary Table: {library_table}")
print(f"Registry Table: {registry_table}")


In [ ]:
# Cell 5: Get Current Version Data (as starting point for changes)
# In production, changes would come from UI/API via staging table
print(f"\nFetching current version data for DTA {dta_id}...")

changes_df = get_dta_current_version(
    spark=spark,
    library_table=library_table,
    registry_table=registry_table,
    dta_id=dta_id
)

if changes_df is None or changes_df.count() == 0:
    raise ValueError(f"No current version found for DTA {dta_id}. Run CREATE_BRANCH first.")

# Remove versioning and audit columns (they'll be added by save_dta_draft)
versioning_cols = ['library_version', 'is_major_version', 'is_dta_major', 
                   'parent_version', 'effective_start_ts', 'effective_end_ts', 'is_current']
audit_cols = ['created_by_principal', 'created_ts', 'last_updated_by_principal', 
              'last_updated_ts', 'databricks_job_id', 'databricks_job_name', 'databricks_run_id']

cols_to_drop = [c for c in versioning_cols + audit_cols if c in changes_df.columns]
changes_df = changes_df.drop(*cols_to_drop)

print(f"Records to save: {changes_df.count()}")


In [ ]:
# Cell 6: Execute SAVE_DRAFT
print(f"\n{'='*80}")
print(f"Saving draft for DTA {dta_id}...")
print(f"{'='*80}")

result_version = save_dta_draft(
    spark=spark,
    library_table=library_table,
    registry_table=registry_table,
    dta_id=dta_id,
    changes_df=changes_df,
    library_type=library_type,
    created_by_principal=created_by_principal,
    databricks_job_id=databricks_job_id,
    databricks_job_name=databricks_job_name,
    databricks_run_id=databricks_run_id
)

print(f"\nDraft saved successfully!")
print(f"New version tag: {result_version}")


In [ ]:
# Cell 7: Set Task Values for Downstream
dbutils.jobs.taskValues.set(key="result_version", value=result_version)
dbutils.jobs.taskValues.set(key="dta_id", value=dta_id)
dbutils.jobs.taskValues.set(key="action", value="SAVE_DRAFT")

print(f"\nTask values set:")
print(f"  result_version: {result_version}")
print(f"  dta_id: {dta_id}")
print(f"  action: SAVE_DRAFT")
